In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchsummaryX import summary
import matplotlib.pyplot as plt
import time
import os

from UnarySim.sw.kernel.nn_utils import *

In [3]:
cwd = os.getcwd()
print(cwd)

d:\project\Anaconda3\Lib\site-packages\UnarySim\sw\test\mlp


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"
print(device)

cpu


In [5]:
# MNIST data loader
transform=transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root=cwd+'/../data/mnist', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=4)

testset = torchvision.datasets.MNIST(root=cwd+'/../data/mnist', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, num_workers=4)

In [6]:
# model = LeNet()
# model = LeNet_clamp()
model = MLP3()
# model = MLP3_tanh()
# model = MLP3_clamp()
# model = MLP3_clamp_train()
model.to(device)
summary(model, torch.zeros((1, 1, 32, 32)).to(device))

      Kernel Shape Output Shape  Params  Mult-Adds
Layer                                             
0_fc1  [1024, 512]     [1, 512]  524800     524288
1_fc2   [512, 512]     [1, 512]  262656     262144
2_fc3    [512, 10]      [1, 10]    5130       5120
--------------------------------------------------
                      Totals
Total params          792586
Trainable params      792586
Non-trainable params       0
Mult-Adds             791552


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_fc1,"[1024, 512]","[1, 512]",524800,524288
1_fc2,"[512, 512]","[1, 512]",262656,262144
2_fc3,"[512, 10]","[1, 10]",5130,5120


In [7]:
bitwidth = 8
clipper = NN_SC_Weight_Clipper(bitwidth=bitwidth)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [9]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
    print(model.fc1.weight.max())
    print(model.fc1.weight.min())
    print(model.fc2.weight.max())
    print(model.fc2.weight.min())
    print(model.fc3.weight.max())
    print(model.fc3.weight.min())
#     model.apply(clipper)
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Train - Epoch %d, Loss: %f, Test Accuracy: %f %%' \
          % (epoch, loss.detach().cpu().item(), 100 * correct / total))

print('Finished Training')

tensor(0.0691, grad_fn=<MaxBackward1>)
tensor(-0.0669, grad_fn=<MinBackward1>)
tensor(0.0865, grad_fn=<MaxBackward1>)
tensor(-0.0809, grad_fn=<MinBackward1>)
tensor(0.0671, grad_fn=<MaxBackward1>)
tensor(-0.0834, grad_fn=<MinBackward1>)
Train - Epoch 0, Loss: 1.588135, Test Accuracy: 91.180000 %
tensor(0.0856, grad_fn=<MaxBackward1>)
tensor(-0.0819, grad_fn=<MinBackward1>)
tensor(0.1105, grad_fn=<MaxBackward1>)
tensor(-0.1088, grad_fn=<MinBackward1>)
tensor(0.0774, grad_fn=<MaxBackward1>)
tensor(-0.0987, grad_fn=<MinBackward1>)
Train - Epoch 1, Loss: 1.595011, Test Accuracy: 92.510000 %
tensor(0.0952, grad_fn=<MaxBackward1>)
tensor(-0.0960, grad_fn=<MinBackward1>)
tensor(0.1314, grad_fn=<MaxBackward1>)
tensor(-0.1315, grad_fn=<MinBackward1>)
tensor(0.0842, grad_fn=<MaxBackward1>)
tensor(-0.1105, grad_fn=<MinBackward1>)
Train - Epoch 2, Loss: 1.536012, Test Accuracy: 93.450000 %
tensor(0.1055, grad_fn=<MaxBackward1>)
tensor(-0.1121, grad_fn=<MinBackward1>)
tensor(0.1484, grad_fn=<MaxBac

In [10]:
model_path = cwd+"\saved_model_state_dict"+"_"+str(bitwidth)+"_no_clamp_new"
torch.save(model.state_dict(), model_path)

# test load from state_dict

In [11]:
model = MLP3_clamp_eval()
model.load_state_dict(torch.load(model_path))
model.eval()
model.to(device)
# model.apply(clipper)
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 80.880000 %


In [12]:
print(model.fc1.weight.max())
print(model.fc1.weight.min())
print(model.fc1.bias.max())
print(model.fc1.bias.min())

print(model.fc2.weight.max())
print(model.fc2.weight.min())
print(model.fc2.bias.max())
print(model.fc2.bias.min())

print(model.fc3.weight.max())
print(model.fc3.weight.min())
print(model.fc3.bias.max())
print(model.fc3.bias.min())

tensor(0.1683, grad_fn=<MaxBackward1>)
tensor(-0.2285, grad_fn=<MinBackward1>)
tensor(0.0785, grad_fn=<MaxBackward1>)
tensor(-0.0576, grad_fn=<MinBackward1>)
tensor(0.2493, grad_fn=<MaxBackward1>)
tensor(-0.2337, grad_fn=<MinBackward1>)
tensor(0.0807, grad_fn=<MaxBackward1>)
tensor(-0.0598, grad_fn=<MinBackward1>)
tensor(0.1396, grad_fn=<MaxBackward1>)
tensor(-0.2298, grad_fn=<MinBackward1>)
tensor(0.0312, grad_fn=<MaxBackward1>)
tensor(-0.0342, grad_fn=<MinBackward1>)


In [13]:
# model.fc1.weight.data = model.fc1.weight*4
# model.fc1.bias.data = model.fc1.bias*4

# model.fc2.weight.data = model.fc2.weight*4
# model.fc2.bias.data = model.fc2.bias*4

# model.fc3.weight.data = model.fc3.weight*4
# model.fc3.bias.data = model.fc3.bias*4


In [14]:
# model_path = cwd+"\saved_model_state_dict"+"_"+str(bitwidth)+"_no_clamp_new"
# torch.save(model.state_dict(), model_path)

In [15]:
# model = MLP3_clamp_eval()
# model.load_state_dict(torch.load(model_path))
# model.eval()
# model.to(device)
# # model.apply(clipper)
# correct = 0
# total = 0
# with torch.no_grad():
#     for data in testloader:
#         images, labels = data[0].to(device), data[1].to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print('Accuracy of the network on the 10000 test images: %f %%' % (
#     100 * correct / total))

In [16]:
# print(model.fc1.weight.max())
# print(model.fc1.weight.min())
# print(model.fc1.bias.max())
# print(model.fc1.bias.min())

# print(model.fc2.weight.max())
# print(model.fc2.weight.min())
# print(model.fc2.bias.max())
# print(model.fc2.bias.min())

# print(model.fc3.weight.max())
# print(model.fc3.weight.min())
# print(model.fc3.bias.max())
# print(model.fc3.bias.min())